In [1]:
import pandas as pd
import numpy as np
import clickhouse_connect
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
import time
from datetime import datetime
from dateutil.tz import tzutc, tzlocal


In [2]:
clickhouse_client = clickhouse_connect.get_client(
    host="clickhouse.matter.bytebeam.io",
    port=443,
    username="matter_rw",
    password="risazAza9saNGEb8",
    secure=True,
    query_limit=0)


In [3]:
utc = datetime.now(tzutc())
index = clickhouse_client.query(f"SELECT id,timestamp,DTU_Phase_A_Current_896,DTU_Phase_B_Current_896,DTU_Phase_C_Current_896,DTU_Resultant_Current_896,DTU_throttle_position_897,DTU_dc_link_voltg_897,DTU_Motor_RPM_897,DTU_ctrl_board_tempr_1_898,DTU_ctrl_board_tempr_2_898,DTU_pwr_board_tempr_1_898,DTU_pwr_board_tempr_2_898,DTU_motor_tempr_1_899,DTU_motor_tempr_2_899,DTU_mcu_tempr_899,DTU_Inverter_control_mode_516,DTU_q_axis_voltg_900,DTU_d_axis_voltg_900,DTU_q_axis_curr_901,DTU_d_axis_curr_901,Id_reference_922,Iq_reference_922,BMS_pack_current_768,BMS_pack_vtg_768,BMS_fet_state_773,BMS_soc_percentage_774,BMS_main_state_773,DTU_regen_status_897,BMS_bat_highest_cell_temp_769,BMS_pump_flow_rate_795,Vehicle_Speed_actual_921,DTU_Rotor_Angle_901,DTU_under_voltg_fault_516,DTU_over_voltg_fault_516,DTU_motor_over_tempr_fault_516,DTU_over_tempr_fault_516,DTU_high_rpm_fault_516,DTU_over_curr_fault_516,DTU_short_circuit_fault_516,DTU_position_sensor_fault_516,BMS_afe_sc_detetction_fault_512,BMS_afe_oc_dchg_fault_512,BMS_afe_oc_fault_512,BMS_afe_otf_fault_512,BMS_ddsg_fault_512,BMS_sop_watt_774,rx_gear_position_920,Ride_sub_modes_848,DTU_brake_switch_status_897,DTU_kill_switch_status_897,DTU_clutch_switch_status_897,DTU_launch_act_status_897,DTU_pam_act_status_897,DTU_deacc_act_status_897,DTU_acc_act_status_897 FROM bytebeam.can_parsed_dbc_base_v0 WHERE timestamp >= now() - interval 30 minute ORDER BY id, timestamp")
arr_index = index.result_rows
index_df1 = pd.DataFrame(arr_index)
index_df1.columns = index.column_names
print(index_df1)

if index_df1.empty:
    formatted_time_utc = time.strftime("%Y-%m-%d %H:%M:%S")
    utc_time = datetime.strptime(formatted_time_utc, "%Y-%m-%d %H:%M:%S")
    local_time = utc_time.astimezone(tzlocal())
    print('No data found at Time: ' + str(local_time))


       id               timestamp  DTU_Phase_A_Current_896  \
0      18 2023-10-05 09:08:49.070                      NaN   
1      18 2023-10-05 09:08:49.170                      NaN   
2      18 2023-10-05 09:08:49.284                      NaN   
3      18 2023-10-05 09:08:49.371                      NaN   
4      18 2023-10-05 09:08:49.489                      NaN   
...    ..                     ...                      ...   
53080  22 2023-10-05 09:38:42.237                  -2320.0   
53081  22 2023-10-05 09:38:42.337                  -1778.0   
53082  22 2023-10-05 09:38:42.438                  -1186.0   
53083  22 2023-10-05 09:38:42.538                   -349.0   
53084  22 2023-10-05 09:38:42.638                      NaN   

       DTU_Phase_B_Current_896  DTU_Phase_C_Current_896  \
0                          NaN                      NaN   
1                          NaN                      NaN   
2                          NaN                      NaN   
3                  

In [4]:
grouped = index_df1.groupby('id')
grouped
index_df = {} 
for group_name, group_data in grouped:
    df_name = f'df_{group_name}'
    index_df[df_name] = group_data
index_df = pd.concat(index_df.values(), ignore_index=True)


In [5]:
Fault_summary = pd.DataFrame(columns=['Index', 'Timestamp', 'Fault Type', 'Device ID'])

inside_fault = False
previous_fault_type = None 
utc = datetime.now(tzutc())

fault_conditions = {
    'DTU_over_voltg_fault_516': 'Voltage Overload',
    'DTU_under_voltg_fault_516': 'Voltage Underload',
    'DTU_motor_over_tempr_fault_516': 'Motor Overtemperature',
    'DTU_over_tempr_fault_516': 'Overtemperature',
    'DTU_high_rpm_fault_516': 'High RPM',
    'DTU_over_curr_fault_516': 'Overcurrent',
    'DTU_short_circuit_fault_516': 'Short Circuit',
    'DTU_position_sensor_fault_516': 'Position Sensor',
    'BMS_ddsg_fault_512': 'DDSG Fault',
    'BMS_afe_oc_fault_512': 'AFE Overcurrent',
    'BMS_afe_otf_fault_512': 'AFE Overtemperature Fast',
    'BMS_afe_oc_dchg_fault_512': 'AFE Overcurrent Discharge',
    'BMS_afe_sc_detetction_fault_512': 'AFE Short Circuit Detection'
}

for index, row in index_df.iterrows():
    for condition, fault_type in fault_conditions.items():
        if row[condition] in ['ON', 'ACTIVE']:
            if not inside_fault or fault_type != previous_fault_type:
                fault_start_index = index 
                fault_start_time = row['timestamp']
                device_id = row['id'] 
                print(f"{fault_type} fault starts at index {index} - Device ID: {device_id}")
                inside_fault = True
                fault_summary_data = [] 
                fault_summary_data.append({'Index': fault_start_index, 'Timestamp': row['timestamp'], 'Fault Type': fault_type, 'Device ID': device_id})
            previous_fault_type = fault_type
            break
    else:
        if inside_fault:
            fault_end_index = index - 1
            fault_end_time = row['timestamp']
            print(f"Fault ends at index {fault_end_index}")
            inside_fault = False
            Fault_summary = pd.concat([Fault_summary, pd.DataFrame(fault_summary_data)], ignore_index=True)


if inside_fault:
    print(f"{fault_type} fault ends at the end of the data at index {index}")
    if fault_summary_data:
        Fault_summary = pd.concat([Fault_summary, pd.DataFrame(fault_summary_data)], ignore_index=True)

if not inside_fault:
    formatted_time_utc = time.strftime("%Y-%m-%d %H:%M:%S")
    utc_time = datetime.strptime(formatted_time_utc, "%Y-%m-%d %H:%M:%S")
    local_time = utc_time.astimezone(tzlocal())
    print('No Faults found at Time: ' + str(local_time))

print(Fault_summary)


Voltage Underload fault starts at index 39980 - Device ID: 22
Fault ends at index 39988
No Faults found at Time: 2023-10-05 15:09:07+05:30
   Index               Timestamp         Fault Type Device ID
0  39980 2023-10-05 09:10:39.739  Voltage Underload        22


In [6]:
row_range = timedelta(seconds=30)
parent_table =pd.DataFrame()
utc = datetime.now(tzutc())
 
for i in range(len(Fault_summary)):    
    fault_start_timestamp = Fault_summary.loc[i,'Timestamp']
    desired_id =Fault_summary.loc[i,'Device ID']
    start_time = fault_start_timestamp - row_range
    end_time = fault_start_timestamp + row_range
    index_pt=clickhouse_client.query(f"SELECT id,timestamp,DTU_Phase_A_Current_896,DTU_Phase_B_Current_896,DTU_Phase_C_Current_896,DTU_Resultant_Current_896,DTU_throttle_position_897,DTU_dc_link_voltg_897,DTU_Motor_RPM_897,DTU_ctrl_board_tempr_1_898,DTU_ctrl_board_tempr_2_898,DTU_pwr_board_tempr_1_898,DTU_pwr_board_tempr_2_898,DTU_motor_tempr_1_899,DTU_motor_tempr_2_899,DTU_mcu_tempr_899,DTU_Inverter_control_mode_516,DTU_q_axis_voltg_900,DTU_d_axis_voltg_900,DTU_q_axis_curr_901,DTU_d_axis_curr_901,Id_reference_922,Iq_reference_922,BMS_pack_current_768,BMS_pack_vtg_768,BMS_fet_state_773,BMS_soc_percentage_774,BMS_main_state_773,DTU_regen_status_897,BMS_bat_highest_cell_temp_769,BMS_pump_flow_rate_795,Vehicle_Speed_actual_921,DTU_Rotor_Angle_901,DTU_under_voltg_fault_516,DTU_over_voltg_fault_516,DTU_motor_over_tempr_fault_516,DTU_over_tempr_fault_516,DTU_high_rpm_fault_516,DTU_over_curr_fault_516,DTU_short_circuit_fault_516,DTU_position_sensor_fault_516,BMS_afe_sc_detetction_fault_512,BMS_afe_oc_dchg_fault_512,BMS_afe_oc_fault_512,BMS_afe_otf_fault_512,BMS_ddsg_fault_512,BMS_sop_watt_774,rx_gear_position_920,Ride_sub_modes_848,DTU_brake_switch_status_897,DTU_kill_switch_status_897,DTU_clutch_switch_status_897,DTU_launch_act_status_897,DTU_pam_act_status_897,DTU_deacc_act_status_897,DTU_acc_act_status_897 FROM bytebeam.can_parsed_dbc_base_v0 WHERE id = '{str(desired_id)}' AND timestamp >= toDateTime64('{str(start_time)}',3,'UTC') AND timestamp <= toDateTime64('{str(end_time)}',3,'UTC') ORDER BY id, timestamp")
    arr_index_pt=index_pt.result_rows
    index_df1_pt = pd.DataFrame(arr_index_pt)
    index_df1_pt.columns=index_pt.column_names
    index_df1_pt    
    parent_table1 = parent_table.append(index_df1_pt, ignore_index=True)
    
    
if not inside_fault:
    formatted_time_utc = time.strftime("%Y-%m-%d %H:%M:%S")
    utc_time = datetime.strptime(formatted_time_utc, "%Y-%m-%d %H:%M:%S")
    local_time = utc_time.astimezone(tzlocal())
    print('No Faults found at Time: ' + str(local_time))
    
print(parent_table1)


No Faults found at Time: 2023-10-05 15:09:13+05:30
     id               timestamp  DTU_Phase_A_Current_896  \
0    22 2023-10-05 09:10:09.807                    -86.0   
1    22 2023-10-05 09:10:09.906                    -37.0   
2    22 2023-10-05 09:10:10.006                     61.0   
3    22 2023-10-05 09:10:10.108                     12.0   
4    22 2023-10-05 09:10:10.206                    -37.0   
..   ..                     ...                      ...   
297  22 2023-10-05 09:10:39.507                     12.0   
298  22 2023-10-05 09:10:39.605                      NaN   
299  22 2023-10-05 09:10:39.739                     12.0   
300  22 2023-10-05 09:10:39.799                      NaN   
301  22 2023-10-05 09:10:39.849                    -86.0   

     DTU_Phase_B_Current_896  DTU_Phase_C_Current_896  \
0                       37.0                     49.0   
1                       37.0                      0.0   
2                      -61.0                      0.0   


In [7]:
dtu_monitoring=pd.DataFrame()
utc = datetime.now(tzutc())

clickhouse_client.insert_df('bytebeam.dtu_monitoring', parent_table1, column_names=["id","timestamp","DTU_Phase_A_Current_896","DTU_Phase_B_Current_896","DTU_Phase_C_Current_896","DTU_Resultant_Current_896","DTU_throttle_position_897","DTU_dc_link_voltg_897","DTU_Motor_RPM_897","DTU_ctrl_board_tempr_1_898","DTU_ctrl_board_tempr_2_898","DTU_pwr_board_tempr_1_898","DTU_pwr_board_tempr_2_898","DTU_motor_tempr_1_899","DTU_motor_tempr_2_899","DTU_mcu_tempr_899","DTU_Inverter_control_mode_516","DTU_q_axis_voltg_900","DTU_d_axis_voltg_900","DTU_q_axis_curr_901","DTU_d_axis_curr_901","Id_reference_922","Iq_reference_922","BMS_pack_current_768","BMS_pack_vtg_768","BMS_fet_state_773","BMS_soc_percentage_774","BMS_main_state_773","DTU_regen_status_897","BMS_bat_highest_cell_temp_769","BMS_pump_flow_rate_795","Vehicle_Speed_actual_921","DTU_Rotor_Angle_901","DTU_under_voltg_fault_516","DTU_over_voltg_fault_516","DTU_motor_over_tempr_fault_516","DTU_over_tempr_fault_516","DTU_high_rpm_fault_516","DTU_over_curr_fault_516","DTU_short_circuit_fault_516","DTU_position_sensor_fault_516","BMS_afe_sc_detetction_fault_512","BMS_afe_oc_dchg_fault_512","BMS_afe_oc_fault_512","BMS_afe_otf_fault_512","BMS_ddsg_fault_512","BMS_sop_watt_774","rx_gear_position_920","Ride_sub_modes_848","DTU_brake_switch_status_897","DTU_kill_switch_status_897","DTU_clutch_switch_status_897","DTU_launch_act_status_897","DTU_pam_act_status_897","DTU_deacc_act_status_897","DTU_acc_act_status_897"])
if dtu_monitoring_v1.empty:
    formatted_time_utc = time.strftime("%Y-%m-%d %H:%M:%S")
    utc_time = datetime.strptime(formatted_time_utc, "%Y-%m-%d %H:%M:%S")
    local_time = utc_time.astimezone(tzlocal())
    print('No Faults found at Time: ' + str(local_time))
    
else:
    print("Updated_dtu_monitoring_v1 table at" + str(local_time))